## adversarial code

In [1]:
DATA_DIR = '/store/dataset/ChestXray-NIHCC/images'
IMAGE_PATH = '/store/dataset/ChestXray-NIHCC/images/00017046_015.png'
#gt = torch.FloatTensor([[0,0,0,0,0,0,0,0,0,0,0,0,1,0]]).cuda()
from PIL import Image
import torch
img = Image.open(IMAGE_PATH).convert('RGB')



import torchvision.transforms as transforms
normalize = transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])
# transform=transforms.Compose([
#     transforms.Resize(256),
#     transforms.TenCrop(224),
#     transforms.Lambda
#     (lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
#     transforms.Lambda
#     (lambda crops: torch.stack([normalize(crop) for crop in crops]))
# ])

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

img_torch = transform(img)
img_torch.shape



torch.Size([3, 224, 224])

In [2]:

import torchvision
import torch.nn as nn

class DenseNet121(nn.Module):
    """Model modified.

    The architecture of our model is the same as standard DenseNet121
    except the classifier layer which has an additional sigmoid function.

    """
    def __init__(self, out_size):
        super(DenseNet121, self).__init__()
        self.densenet121 = torchvision.models.densenet121(pretrained=True)
        num_ftrs = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_ftrs, out_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.densenet121(x)
        return x

In [3]:
import torch.backends.cudnn as cudnn
import torch
import re
import os

CKPT_PATH = '/home/lrh/program/git/pytorch-example/CheXNet/model.pth.tar'
N_CLASSES = 14

cudnn.benchmark = True

# initialize and load the model
model = DenseNet121(N_CLASSES).cuda()
model = torch.nn.DataParallel(model).cuda()
model.eval()

if os.path.isfile(CKPT_PATH):
    print("=> loading checkpoint")
    checkpoint = torch.load(CKPT_PATH)

    pattern = re.compile(
        r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')
    state_dict = checkpoint['state_dict']
    for key in list(state_dict.keys()):
        res = pattern.match(key)
        if res:
            new_key = res.group(1) + res.group(2)
            state_dict[new_key] = state_dict[key]
            del state_dict[key]

    model.load_state_dict(state_dict)
    print("=> loaded checkpoint")
else:
    print("=> no checkpoint found")

=> loading checkpoint
=> loaded checkpoint


In [8]:
import libadver.attack as attack
import torch.nn as nn
import numpy as np
import torchvision
pgd_params = {
    'ord': np.inf,
    'y': None,
    'y_target': None,
    'eps': 5.0 / 255,
    'eps_iter': 0.5 / 255,
    'nb_iter': 40,
    'rand_init': True,
    'rand_minmax': 5.0 / 255,
    'clip_min': 0.,
    'clip_max': 1.,
    'sanity_checks': True,
    'criterion' : nn.BCELoss()
}

pgdAttack = attack.ProjectGradientDescent(model)

pgd_params["clip_min"] = torch.min(img_torch)
pgd_params["clip_max"] = torch.max(img_torch)

targeted = torch.FloatTensor([[0,0,0,0,0,0,0,0,0,0,0,0,0,0]]).cuda()
pgd_params["y_target"] = targeted

img = img_torch.unsqueeze(0).cuda()
adv_x = pgdAttack.generate(img, **pgd_params)

torchvision.utils.save_image(adv_x, "adv_x.png", normalize = True)
torchvision.utils.save_image(img, "img.png", normalize = True)
torchvision.utils.save_image(adv_x - img, "delta_m.png", normalize = True)
x_pred = model(img)
adv_pred = model(adv_x)

print(x_pred)
print(adv_pred)

tensor([[0.2656, 0.0708, 0.8437, 0.5624, 0.6976, 0.5904, 0.4092, 0.4449, 0.3253,
         0.0787, 0.4114, 0.8049, 0.9772, 0.2655]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
tensor([[6.0785e-05, 2.0378e-06, 3.3695e-06, 3.1833e-04, 2.7117e-04, 4.3988e-04,
         3.6961e-05, 2.9203e-04, 2.0045e-04, 3.8790e-06, 4.8334e-07, 1.1652e-06,
         2.4777e-05, 1.1399e-05]], device='cuda:0', grad_fn=<SigmoidBackward>)


In [ ]:
model = model.eval()
inp = img_torch.unsqueeze(0)
#bs, n_crops, c, h, w = inp.size()

input_var = inp
#print(input_var.shape)
output = model(input_var)
#output_mean = output.view(bs, n_crops, -1).mean(1)
print(output)

In [1]:
gt_path = "gt.npy"
pred_path = "pred.npy"
from sklearn.metrics import roc_auc_score
import numpy as np
gt = np.load(gt_path)
pred = np.load(pred_path)
#print(gt[0:10])
#print(pred[0:10])
AUROCs = []
for i in range(14):
    AUROCs.append(roc_auc_score(gt[:, i], pred[:, i]))
    print(AUROCs[i])
AUROC_avg = np.array(AUROCs).mean()
print('The average AUROC is {AUROC_avg:.3f}'.format(AUROC_avg=AUROC_avg))
ACCs = []
for i in range(14):
    predictLabels = pred[:, i] > 0.5
    #print(gt[:, i] == predictLabels)
    acc = (gt[:, i] == predictLabels).sum() / gt.shape[0]
    #print(acc)
    ACCs.append(acc)
print(ACCs)
print(np.array(ACCs).mean())



0.8232972431555852
0.9129112651587029
0.8823640305627175
0.7082440712910067
0.8528536562120591
0.778697148282156
0.7738251789218398
0.8664688403043657
0.8086007602372302
0.8914294840921636
0.9100857525676305
0.8224918776695301
0.7774652280589532
0.90514364827705
The average AUROC is 0.837
[0.8087192974635582, 0.8394775553871529, 0.8197744394418937, 0.6342887710069987, 0.8795970222440155, 0.819194936031739, 0.743502875228458, 0.7975304239290332, 0.735345250300896, 0.8071590959746802, 0.797084652075068, 0.7896848392992466, 0.7266526991485758, 0.9099986626844381]
0.7934293228725539


In [2]:
gt_path = "gt.npy"
pred_path = "pred_advx.npy"
from sklearn.metrics import roc_auc_score
import numpy as np
gt = np.load(gt_path)
pred = np.load(pred_path)
#print(gt[0:10])
#print(pred[0:10])
AUROCs = []
for i in range(14):
    AUROCs.append(roc_auc_score(gt[:, i], pred[:, i]))
    print(AUROCs[i])
AUROC_avg = np.array(AUROCs).mean()
print('The average AUROC is {AUROC_avg:.3f}'.format(AUROC_avg=AUROC_avg))
ACCs = []
for i in range(14):
    predictLabels = pred[:, i] > 0.5
    #print(gt[:, i] == predictLabels)
    acc = (gt[:, i] == predictLabels).sum() / gt.shape[0]
    #print(acc)
    ACCs.append(acc)
print(ACCs)
print(np.array(ACCs).mean())

0.1907986255215102
0.1694429281351157
0.2555598234008782
0.18958887080650325
0.10203017457495783
0.1579611181349884
0.099630331855927
0.14863915752179008
0.15201406814498572
0.22256313322909174
0.03697507087351949
0.13559066408234
0.14459860340881364
0.13130700898107445
The average AUROC is 0.153
[0.10631658717068604, 0.7457763116836803, 0.5614050728836981, 0.16150314269157046, 0.06089243525163821, 0.2571212053670931, 0.06454776445415236, 0.063388757633843, 0.04965898453171667, 0.5536040654393082, 0.04836624615521776, 0.3001827664601257, 0.109347835777649, 0.3017429679490037]
0.24170386738924157


In [ ]:
gt_path = "../gt_no_tencrop.npy"
pred_path = "../pred_no_tencrop.npy"
from sklearn.metrics import roc_auc_score
import numpy as np
gt = np.load(gt_path)
pred = np.load(pred_path)
#print(gt[0:10])
#print(pred[0:10])
AUROCs = []
for i in range(14):
    AUROCs.append(roc_auc_score(gt[:, i], pred[:, i]))
    print(AUROCs[i])
AUROC_avg = np.array(AUROCs).mean()
print('The average AUROC is {AUROC_avg:.3f}'.format(AUROC_avg=AUROC_avg))
ACCs = []
for i in range(14):
    predictLabels = pred[:, i] > 0.5
    #print(gt[:, i] == predictLabels)
    acc = (gt[:, i] == predictLabels).sum() / gt.shape[0]
    #print(acc)
    ACCs.append(acc)
print(ACCs)
print(np.array(ACCs).mean())



## prediction co-occurrence matrix

In [3]:
import numpy as np
np.set_printoptions(suppress=True, threshold=np.nan)
#pred_path = "../pred_no_tencrop.npy"
pred_path = "pred_advx.npy"
pred = np.load(pred_path)

predLabels = (pred>0.5).astype(np.int)
gt = predLabels

#print(gt.shape)
labelNum = gt.shape[1]
coMatrix = np.zeros([labelNum,labelNum])

for i in range(14):
    sourGt = gt[:,i]
    coMatrix[i,i] = (gt[gt[:,i] == 1,:].sum(axis=1) == 1).sum()

    for j in range(i+1,14):
        desGt = gt[:,j]
        coMatrix[i,j] = ((sourGt + desGt) == 2).sum()
        coMatrix[j,i] = coMatrix[i,j]

coMatrix = coMatrix[0:8,0:8]
np.save("coMatrix_predadvx.npy", coMatrix)

## calculate co-occurrence matrix

In [ ]:
gt_path = "../gt.npy"
import numpy as np
np.set_printoptions(suppress=True, threshold=np.nan)
gt = np.load(gt_path)

coMatrix = np.zeros([14,14])

print(coMatrix)

for i in range(14):
    sourGt = gt[:,i] 
    for j in range(i,14):
        desGt = gt[:,j]
        coMatrix[i,j] = ((sourGt + desGt) == 2).sum()
print(coMatrix)

## read labels

In [ ]:
import numpy as np
labels = []
image_list_file = "../ChestX-ray14/labels/train_list.txt"
with open(image_list_file, "r") as f:
    for line in f:
        items = line.split()
        label = items[1:]
        label = [int(i) for i in label]
        labels.append(label)
gt = np.array(labels)

coMatrix = np.zeros([14,14])

print(coMatrix)




for i in range(14):
    sourGt = gt[:,i]
    for j in range(i,14):
        desGt = gt[:,j]
        coMatrix[i,j] = ((sourGt + desGt) == 2).sum()
print(coMatrix)

# merge csv file

In [ ]:
trainPath = "../ChestX-ray14/labels/train_list.txt"
testPath = "../ChestX-ray14/labels/test_list.txt"
valPath = "../ChestX-ray14/labels/val_list.txt"
labels = []
with open(trainPath, "r") as f:
    for line in f:
        items = line.split()
        label = items[1:]
        label = [int(i) for i in label]
        labels.append(label)
with open(testPath, "r") as f:
    for line in f:
        items = line.split()
        label = items[1:]
        label = [int(i) for i in label]
        labels.append(label)

with open(valPath, "r") as f:
    for line in f:
        items = line.split()
        label = items[1:]
        label = [int(i) for i in label]
        labels.append(label)

gt = np.array(labels)
print(gt.shape)

print((gt[gt[:,0] == 1,:].sum(axis=1) == 1 ).sum())

coMatrix = np.zeros([14,14])

for i in range(14):
    sourGt = gt[:,i] 
    for j in range(i,14):
        desGt = gt[:,j]
        coMatrix[i,j] = ((sourGt + desGt) == 2).sum()
        coMatrix[j,i] = coMatrix[i,j]
print(coMatrix)

print(coMatrix.sum(axis=1))

## compute co-occurrence statistics

In [ ]:
import numpy as np
def computeCoccurrenceMatrix(fileList):
    labels = []
    for filename in fileList:
        with open(filename, "r") as f:
            for line in f:
                items = line.split()
                label = items[1:]
                label = [int(i) for i in label]
                labels.append(label)
    gt = np.array(labels)
    #print(gt.shape)
    labelNum = gt.shape[1]
    coMatrix = np.zeros([labelNum,labelNum])

    for i in range(14):
        sourGt = gt[:,i]
        coMatrix[i,i] = (gt[gt[:,i] == 1,:].sum(axis=1) == 1).sum()
        
        for j in range(i+1,14):
            desGt = gt[:,j]
            coMatrix[i,j] = ((sourGt + desGt) == 2).sum()
            coMatrix[j,i] = coMatrix[i,j]
            
    return coMatrix

# fileList = ["../ChestX-ray14/labels/train_list.txt",
#             "../ChestX-ray14/labels/test_list.txt",
#             "../ChestX-ray14/labels/val_list.txt"]
# file with form:
#     [image_name, labels(0 1 1 0 0 0 0...)]
fileList = ["../ChestX-ray14/labels/test_list.txt"]
coMatrix = computeCoccurrenceMatrix(fileList)

np.save("coMatrix_test.npy", coMatrix)

In [ ]:
np.set_printoptions(suppress=True, threshold=np.nan)
np.load("coMatrix_test.npy")

In [ ]:
import numpy as np
def computeCoccurrenceMatrix(fileList):
    labels = []
    for filename in fileList:
        with open(filename, "r") as f:
            for line in f:
                items = line.split()
                label = items[1:]
                label = [int(i) for i in label]
                labels.append(label)
    gt = np.array(labels)
    #print(gt.shape)
    labelNum = gt.shape[1]
    coMatrix = np.zeros([labelNum,labelNum])

    for i in range(14):
        sourGt = gt[:,i]
        coMatrix[i,i] = (gt[gt[:,i] == 1,:].sum(axis=1) == 1).sum()
        
        for j in range(i+1,14):
            desGt = gt[:,j]
            coMatrix[i,j] = ((sourGt + desGt) == 2).sum()
            coMatrix[j,i] = coMatrix[i,j]
            
    return coMatrix

# fileList = ["../ChestX-ray14/labels/train_list.txt",
#             "../ChestX-ray14/labels/test_list.txt",
#             "../ChestX-ray14/labels/val_list.txt"]
# file with form:
#     [image_name, labels(0 1 1 0 0 0 0...)]
fileList = ["../ChestX-ray14/labels/test_list.txt"]
coMatrix = computeCoccurrenceMatrix(fileList)[0:8,0:8]

np.save("coMatrix_test8.npy", coMatrix)

In [15]:
import torchvision.models as models
vgg16 = models.vgg16_bn()

#print(vgg16)
list(vgg16.features.children())[0:6]




resnet50 = models.resnet50()

layer1 = resnet50.layer1

#print(*list(resnet50.children())[0:4])

print(resnet50)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F